# Introduction

I wanted a numpy database to test and prototype this kit. I mean I have GPDK45nm but it has very few data points to see any trends properly. [NCSU's FreePDK45](https://eda.ncsu.edu/freepdk/freepdk45/) on the other hand has even corner models! But the problem is that they are HSPICE and since I have primarily written my importer function for Cadence Spectre I did not consider FreePDK45 as an option.

Luckily [Mohamed Watfa's gmid toolkit in python](https://github.com/medwatt/gmid) recently published an [update](https://github.com/medwatt/gmid/commit/a5f52b4c1070470944da977b1d8a6bee99118a3c) which finally removed pyspice dependency which now enables me to seamlessly use the [LookupTableGenerator](https://github.com/medwatt/gmid#generating-a-lookup-table). Now I need to convert it to a format so that my library can read it. 

In [ ]:
# You need to have the gmid toolkit installed and the model file at the correct location.

# from main.lookup_table_generator import LookupTableGenerator
# import numpy as np
# l_sweep_initial = np.array([45e-9])
# l_sweep_small  = np.arange(100e-9,1e-6,100e-9) # excluded the last value.
# l_sweep_large  = np.arange(1e-6,11e-6,1e-6) # excludes the last value
# l_sweep = np.concatenate((l_sweep_initial,l_sweep_small,l_sweep_large))
# lengths = l_sweep.tolist()

# obj = LookupTableGenerator(
#     description="freepdk 45nm ngspice",
#     simulator="ngspice",
#     model_paths=[
#         "./models/models_ss/NMOS_VTH.inc",
#         "./models/models_ss/PMOS_VTH.inc",
#         ],
#     model_names={
#         "nmos": "NMOS_VTH",
#         "pmos": "PMOS_VTH",
#     },
#     temp=25,
#     vsb=(0, 0.5, 0.1),
#     vgs=(0, 1.0, 0.01),
#     vds=(0, 1.0, 0.01),
#     width=10e-6,
#     lengths=lengths,
# )
# obj.build("./freepdk45/original_npy/ss.npy")

## Post-processing

Surprisingly the Lookup Table (LUT) generation was very fastest I have seen. Now to make the structure compliant with my `.npy` structure I am doing post processing.
Hopefully I don't have to make any major changes.

In [ ]:
import sys
import os
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
# Append the path of your library to the system path
sys.path.append(os.path.abspath("../../"))
from analog_daddy.look_up import look_up
from analog_daddy.utils import pretty_print_structure, describe_structure
from analog_daddy.conf import * # import all the config variables
from analog_daddy.importer import lists_to_arrays

In [ ]:
ORIGINAL_DATABASE_PATH = "../../ignored_folder/databases/freepdk45/original_npy/nom_25.npy"
FINAL_DATABASE_PATH = "../../ignored_folder/databases/freepdk45/NPY/nom_25.npy"
data = lists_to_arrays(np.load(ORIGINAL_DATABASE_PATH, allow_pickle=True).item())
data['corner'] = 'nom'
data['temperature'] = 25

In [ ]:
pretty_print_structure(describe_structure(data))

In [ ]:
# nesting the lengths and width to be accessed without parent structure
# also renaming keys to match the defaults.
data['nmos']['length'] = data['l']
data['pmos']['length'] = data['l']
data['nmos']['w'] = data['w']
data['pmos']['w'] = data['w']

# renaming vgs, vds and vsb to match the defaults.
data['nmos']['gs'] = data['nmos']['vgs']
data['nmos']['ds'] = data['nmos']['vds']
data['nmos']['sb'] = data['nmos']['vsb']
data['pmos']['gs'] = data['pmos']['vgs']
data['pmos']['ds'] = data['pmos']['vds']
data['pmos']['sb'] = data['pmos']['vsb']

del data['l']
del data['w']
del data['nmos']['vgs']
del data['nmos']['vds']
del data['nmos']['vsb']
del data['pmos']['vgs']
del data['pmos']['vds']
del data['pmos']['vsb']

data['info'] = data['description']
del data['description']
del data['parameter_names']
del data['nmos']['model_name']
del data['pmos']['model_name']
data['nmos_vth'] = data['nmos']
data['pmos_vth'] = data['pmos']
del data['nmos']
del data['pmos']

for key in data['pmos_vth'].keys():
    data['pmos_vth'][key] = abs(data['pmos_vth'][key])


In [ ]:
# Restructure the array dimensions
# Original structure seems to be
# cgg[length][sb][gs][ds]
# I need it to be
# cgg[ds][gs][length][sb]
# thus  transposed_array = original_array.transpose(3, 2, 0, 1)

def transpose_4d_arrays(d):
    """
    Recursively transposes 4D arrays within a nested dictionary.
    """
    for key, value in d.items():
        if isinstance(value, np.ndarray) and value.ndim == 4:
            d[key] = value.transpose(3, 2, 0, 1)
        elif isinstance(value, dict):
            transpose_4d_arrays(value)
    return d

new_data = transpose_4d_arrays(data)

In [ ]:
pretty_print_structure(describe_structure(new_data))

In [ ]:
np.save(FINAL_DATABASE_PATH, new_data)
print("Done writing the npy file")